# Cài đặt thư viện


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/

Mounted at /content/drive
/content


In [2]:
!pip install -r /content/drive/MyDrive/KLTN_19521818/osr_closed_set_all_you_need-main/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 KB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 69.0 MB/s eta 0:00:00
  Attempting u

In [3]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from tqdm import tqdm 

from sklearn.metrics import average_precision_score
import numpy as np

In [5]:
import os
import argparse
import datetime
import time
import pandas as pd
import importlib

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.backends.cudnn as cudnn
import sys
import os

py_file_location = " "
sys.path.append(os.path.abspath(py_file_location))
import methods
from methods.ARPL.core import evaluation
#from methods.ARPL.arpl_models.arpl_models import classifier32ABN
from methods.ARPL.arpl_utils import AverageMeter
from methods.ARPL.arpl_models import gan
from methods.ARPL.arpl_utils import save_networks,load_networks
from methods.ARPL.core import train, train_cs
from methods.MSP.test_msp import test
#from methods.ARPL.arpl_models.arpl_models import classifier32
from utils.utils import init_experiment, seed_torch, str2bool, get_default_hyperparameters
from utils.schedulers import get_scheduler
from data.open_set_datasets import get_class_splits, get_datasets
from models.model_utils import get_model

from config import exp_root
from models.classifier32  import classifier32

# Định nghĩa tham số

In [6]:
parser = argparse.ArgumentParser("Training")
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default='cifar-10-10', help="")
parser.add_argument('--out-num', type=int, default=10, help='For cifar-10-100')
parser.add_argument('--image_size', type=int, default=32)

# optimization
parser.add_argument('--optim', type=str, default=None, help="Which optimizer to use {adam, sgd}")
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--lr', type=float, default=0.1, help="learning rate for model")
parser.add_argument('--weight_decay', type=float, default=1e-4, help="LR regularisation on weights")
parser.add_argument('--gan_lr', type=float, default=0.0002, help="learning rate for gan")
parser.add_argument('--max-epoch', type=int, default=50)
parser.add_argument('--scheduler', type=str, default='cosine_warm_restarts')
parser.add_argument('--temp', type=float, default=1.0, help="temp")
parser.add_argument('--num_restarts', type=int, default=2, help='How many restarts for cosine_warm_restarts schedule')
parser.add_argument('--num-centers', type=int, default=1)

# model
parser.add_argument('--loss', type=str, default='Softmax')
parser.add_argument('--weight-pl', type=float, default=0.1, help="weight for center loss")
parser.add_argument('--label_smoothing', type=float, default=None, help="Smoothing constant for label smoothing."
                                                                        "No smoothing if None or 0")
parser.add_argument('--beta', type=float, default=0.1, help="weight for entropy loss")
parser.add_argument('--model', type=str, default='classifier32')
parser.add_argument('--resnet50_pretrain', type=str, default='places_moco',
                        help='Which pretraining to use if --model=timm_resnet50_pretrained.'
                             'Options are: {iamgenet_moco, places_moco, places}', metavar='BOOL')
parser.add_argument('--feat_dim', type=int, default=128, help="Feature vector dim, only for classifier32 at the moment")

# aug
parser.add_argument('--transform', type=str, default='rand-augment')
parser.add_argument('--rand_aug_m', type=int, default=6)
parser.add_argument('--rand_aug_n', type=int, default=1)

# misc
parser.add_argument('--num_workers', default=4, type=int)
parser.add_argument('--split_train_val', default=False, type=str2bool,
                        help='Subsample training set to create validation set', metavar='BOOL')
parser.add_argument('--use_default_parameters', default=False, type=str2bool,
                    help='Set to True to use optimized hyper-parameters from paper', metavar='BOOL')
parser.add_argument('--device', default='cuda:0', type=str, help='Which GPU to use')
parser.add_argument('--gpus', default=[0], type=int, nargs='+',
                        help='device ids assignment (e.g 0 1 2 3)')
parser.add_argument('--nz', type=int, default=100)
parser.add_argument('--ns', type=int, default=1)
parser.add_argument('--eval-freq', type=int, default=1)
parser.add_argument('--print-freq', type=int, default=100)
parser.add_argument('--checkpt_freq', type=int, default=20)
parser.add_argument('--gpu', type=str, default='0')
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--use-cpu', action='store_true')
parser.add_argument('--eval', action='store_true', help="Eval", default=False)
parser.add_argument('--cs', action='store_true', help="Confusing Sample", default=False)
parser.add_argument('--train_feat_extractor', default=True, type=str2bool,
                        help='Train feature extractor (only implemented for renset_50_faces)', metavar='BOOL')
parser.add_argument('--split_idx', default=0, type=int, help='0-4 OSR splits for each dataset')
parser.add_argument('--use_softmax_in_eval', default=True, type=str2bool,
                        help='Do we use softmax or logits for evaluation', metavar='BOOL')


_StoreAction(option_strings=['--use_softmax_in_eval'], dest='use_softmax_in_eval', nargs=None, const=None, default=True, type=<function str2bool at 0x7fb361278790>, choices=None, help='Do we use softmax or logits for evaluation', metavar='BOOL')

# MỘT SỐ HÀM

In [7]:
def get_optimizer(args, params_list):

    if args.optim is None:

        if options['dataset'] == 'tinyimagenet':
            optimizer = torch.optim.Adam(params_list, lr=args.lr)
        else:
            optimizer = torch.optim.SGD(params_list, lr=args.lr, momentum=0.9, weight_decay=args.weight_decay)

    elif args.optim == 'sgd':

        optimizer = torch.optim.SGD(params_list, lr=args.lr, momentum=0.9, weight_decay=args.weight_decay)

    elif args.optim == 'adam':

        optimizer = torch.optim.Adam(params_list, lr=args.lr)

    else:

        raise NotImplementedError

    return optimizer

In [8]:
def get_mean_lr(optimizer):
    return torch.mean(torch.Tensor([param_group['lr'] for param_group in optimizer.param_groups])).item()

## Hàm train






In [9]:
def train(net, criterion, optimizer, trainloader, epoch=None, **options):
    net.train()
    losses = AverageMeter()

    torch.cuda.empty_cache()

    loss_all = 0
    for batch_idx, (data, labels, idx) in enumerate(tqdm(trainloader)):

        if options['use_gpu']:
            data, labels = data.cuda(), labels.cuda()

        with torch.set_grad_enabled(True):
            optimizer.zero_grad()
            x, y = net(data, True)
            logits, loss = criterion(x, y, labels)
            
            loss.backward()
            optimizer.step()
        
        losses.update(loss.item(), data.size(0))
        
        loss_all += losses.avg

    print("Batch {}/{}\t Loss {:.6f} ({:.6f})".format(batch_idx + 1, len(trainloader), losses.val, losses.avg))

    return loss_all

In [10]:
args = parser.parse_args()

    # ------------------------
    # Update parameters with default hyperparameters if specified
    # ------------------------
if args.use_default_parameters:
        print('NOTE: Using default hyper-parameters...')
        args = get_default_hyperparameters(args)

args.exp_root = exp_root
args.epochs = args.max_epoch
img_size = args.image_size
results = dict()

In [11]:
if args.feat_dim is None:
            args.feat_dim = 128 if args.model == 'classifier32' else 2048

args.train_classes, args.open_set_classes = get_class_splits(args.dataset, args.split_idx,
                                                                     cifar_plus_n=args.out_num)
img_size = args.image_size

args.save_name = '{}_{}_{}'.format(args.model, args.seed, args.dataset)
runner_name = os.path.dirname('/content/drive/MyDrive/CS505/osr_closed_set_all_you_need-main/methods/ARPL/osr.py').split("/")[-2:]
args = init_experiment(args, runner_name=runner_name)

[0]
Experiment saved to: /content/drive/MyDrive/Khóa luận tốt nghiệp _ 19521818_Nguyễn Thị Ly/osr_closed_set_all_you_need-main/methods/ARPL/log/(09.01.2023_|_25.061)
['methods', 'ARPL']
Namespace(batch_size=128, beta=0.1, checkpt_freq=20, cs=False, cuda=True, dataset='cifar-10-10', device=device(type='cuda', index=0), epochs=50, eval=False, eval_freq=1, exp_root='/content/drive/MyDrive/Khóa luận tốt nghiệp _ 19521818_Nguyễn Thị Ly/osr_closed_set_all_you_need-main', f='/root/.local/share/jupyter/runtime/kernel-bf5253a3-b2a6-41f9-997a-7e69d38f9df9.json', feat_dim=128, gan_lr=0.0002, gpu='0', gpus=[0], image_size=32, label_smoothing=None, log_dir='/content/drive/MyDrive/Khóa luận tốt nghiệp _ 19521818_Nguyễn Thị Ly/osr_closed_set_all_you_need-main/methods/ARPL/log/(09.01.2023_|_25.061)', loss='Softmax', lr=0.1, max_epoch=50, model='classifier32', model_dir='/content/drive/MyDrive/Khóa luận tốt nghiệp _ 19521818_Nguyễn Thị Ly/osr_closed_set_all_you_n

In [12]:
seed_torch(args.seed)

# Dataset


In [13]:
#Load data từ drive và chuẩn hỏa dữ liệu

In [14]:
datasets= get_datasets(args.dataset, transform=args.transform, train_classes=args.train_classes,
                                open_set_classes=args.open_set_classes, balance_open_set_eval=True,
                                split_train_val=args.split_train_val, image_size=args.image_size, seed=args.seed,
                                args=args)


Loading datasets...


In [15]:
datasets

{'train': Dataset CustomCIFAR10
     Number of datapoints: 30000
     Root location: /content/drive/MyDrive/KLTN_19521818/osr_closed_set_all_you_need-main/dataset/cifar10
     Split: Train
     StandardTransform
 Transform: Compose(
                Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
                RandomCrop(size=(32, 32), padding=4)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
            ), 'val': Dataset CustomCIFAR10
     Number of datapoints: 400
     Root location: /content/drive/MyDrive/KLTN_19521818/osr_closed_set_all_you_need-main/dataset/cifar10
     Split: Test
     StandardTransform
 Transform: Compose(
                Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
                ToTensor()
                Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
            ), '

In [16]:
  # RANDAUG HYPERPARAM SWEEP
if args.transform == 'rand-augment':
            if args.rand_aug_m is not None:
                if args.rand_aug_n is not None:
                    datasets['train'].transform.transforms[0].m = args.rand_aug_m
                    datasets['train'].transform.transforms[0].n = args.rand_aug_n

In [17]:
 # ------------------------
        # DATALOADER
        # ------------------------
dataloaders = {}
for k, v, in datasets.items():
            shuffle = True if k == 'train' else False
            dataloaders[k] = DataLoader(v, batch_size=args.batch_size,
                                        shuffle=shuffle, sampler=None, num_workers=args.num_workers)

      

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
        # SAVE PARAMS
        # ------------------------
options = vars(args)
options.update(
            {
                'item':     1,
                'known':    args.train_classes,
                'unknown':  args.open_set_classes,
                'img_size': img_size,
                'dataloaders': dataloaders,
                'num_classes': len(args.train_classes)
            }
        )
dir_name = '{}_{}'.format(options['model'], options['loss'])
dir_path = os.path.join('/'.join(args.log_dir.split("/")[:-2]), 'results', dir_name)

if not os.path.exists(dir_path):
            os.makedirs(dir_path)

if options['dataset'] == 'cifar-10-100':
            file_name = '{}_{}.csv'.format(options['dataset'], options['out_num'])
            if options['cs']:
                file_name = '{}_{}_cs.csv'.format(options['dataset'], options['out_num'])
else:
            file_name = options['dataset'] + '.csv'
            if options['cs']:
                file_name = options['dataset'] + 'cs' + '.csv'

print('result path:', os.path.join(dir_path, file_name))
        # ------------------------

result path: /content/drive/MyDrive/Khóa luận tốt nghiệp _ 19521818_Nguyễn Thị Ly/osr_closed_set_all_you_need-main/methods/ARPL/results/classifier32_Softmax/cifar-10-10.csv


In [19]:
torch.manual_seed(options['seed'])
os.environ['CUDA_VISIBLE_DEVICES'] = options['gpu']
use_gpu = torch.cuda.is_available()
if options['use_cpu']: use_gpu = False

if use_gpu:
        print("Currently using GPU: {}".format(options['gpu']))
        cudnn.benchmark = False
        torch.cuda.manual_seed_all(options['seed'])
else:
        print("Currently using CPU")


Currently using GPU: 0


## Chia dữ liệu

In [31]:
trainloader = dataloaders['train']
testloader = dataloaders['val']
outloader = dataloaders['test_unknown']

# Xây dựng model


In [21]:
print("Creating model: {}".format(options['model']))
if args.model == 'classifier32':
            net = classifier32(num_classes=len(args.train_classes), feat_dim=args.feat_dim)
else:
            raise NotImplementedError
 

Creating model: classifier32


In [22]:
net

classifier32(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv9): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [23]:
feat_dim = args.feat_dim

In [24]:
options.update(
        {
            'feat_dim': feat_dim,
            'use_gpu':  use_gpu
        }
    )

    # -----------------------------
    # GET LOSS
    # -----------------------------
Loss = importlib.import_module('methods.ARPL.loss.'+options['loss'])
criterion = getattr(Loss, options['loss'])(**options)

In [25]:
import torch.nn.functional as F

In [26]:
if use_gpu:
        net = nn.DataParallel(net).cuda()
        criterion = criterion.cuda()
       


model_path = os.path.join(args.log_dir, 'arpl_models', options['dataset'])
if not os.path.exists(model_path):
        os.makedirs(model_path)

params_list = [{'params': net.parameters()},
                    {'params': criterion.parameters()}]

In [27]:
net.parameters()

<generator object Module.parameters at 0x7fb2eeb94f90>

In [28]:
optimizer = get_optimizer(args=args, params_list=params_list)

    # -----------------------------
    # GET SCHEDULER
    # ----------------------------
scheduler = get_scheduler(optimizer, args)

start_time = time.time()


In [29]:
def get_curve_online(known, novel, stypes = ['Bas']):
    tp, fp = dict(), dict()
    tnr_at_tpr95 = dict()
    
    for stype in stypes:
        
        known.sort()
        novel.sort()
        #print(len(novel),novel)
        end = np.max([np.max(known), np.max(novel)])
        start = np.min([np.min(known),np.min(novel)])
        num_k = known.shape[0]
        num_n = novel.shape[0]
        
        # 1 MA TRẬN CÓ KÍCH THƯỚC  2 LẦN DỮ LIỆU VỚI KIỂU INT 
        tp[stype] = -np.ones([num_k+num_n+1], dtype=int)
        fp[stype] = -np.ones([num_k+num_n+1], dtype=int)
  
      
        tp[stype][0], fp[stype][0] = num_k, num_n
     
        k, n = 0, 0
        for l in range(num_k+num_n):
            if k == num_k:
                tp[stype][l+1:] = tp[stype][l]
                fp[stype][l+1:] = np.arange(fp[stype][l]-1, -1, -1)
                break
            elif n == num_n:
                tp[stype][l+1:] = np.arange(tp[stype][l]-1, -1, -1)
                fp[stype][l+1:] = fp[stype][l]
                break
            else:
                if novel[n] < known[k]:
                    n += 1
                    tp[stype][l+1] = tp[stype][l]
                    fp[stype][l+1] = fp[stype][l] - 1
                else:
                    k += 1
                    tp[stype][l+1] = tp[stype][l] - 1
                    fp[stype][l+1] = fp[stype][l]
       
        tpr95_pos = np.abs(tp[stype] / num_k - .95).argmin()
        
        tnr_at_tpr95[stype] = 1. - fp[stype][tpr95_pos] / num_n
    return tp, fp, tnr_at_tpr95

In [30]:
def metric_ood(x1, x2, stypes = ['Bas'], verbose=True):
    tp, fp, tnr_at_tpr95 = get_curve_online(x1, x2, stypes)
    results = dict()
    mtypes = ['TNR', 'AUROC', 'DTACC', 'AUIN', 'AUOUT']
   # if verbose:
       # print('      ', end='')
       #3 for mtype in mtypes:
         #   print(' {mtype:6s}'.format(mtype=mtype), end='')
       # print('')
        
    for stype in stypes:
        #3if verbose:
            #print('{stype:5s} '.format(stype=stype), end='')
        results[stype] = dict()
        
        # TNR
        mtype = 'TNR'
        results[stype][mtype] = 100.*tnr_at_tpr95[stype]
        #if verbose:
           # print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUROC
        mtype = 'AUROC'
        
        tpr = np.concatenate([[1.], tp[stype]/tp[stype][0], [0.]])
      
        fpr = np.concatenate([[1.], fp[stype]/fp[stype][0], [0.]])
       
        # tinh dien tic h
        results[stype][mtype] = 100.*(-np.trapz(1.-fpr, tpr))
        #if verbose:
           # print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # DTACC
        mtype = 'DTACC'
        results[stype][mtype] = 100.*(.5 * (tp[stype]/tp[stype][0] + 1.-fp[stype]/fp[stype][0]).max())
       # if verbose:
           # print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUIN
       # mtype = 'AUIN'
        denom = tp[stype]+fp[stype]
        denom[denom == 0.] = -1.
        pin_ind = np.concatenate([[True], denom > 0., [True]])
        pin = np.concatenate([[.5], tp[stype]/denom, [0.]])
        results[stype][mtype] = 100.*(-np.trapz(pin[pin_ind], tpr[pin_ind]))
       # if verbose:
            #print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUOUT
        mtype = 'AUOUT'
        denom = tp[stype][0]-tp[stype]+fp[stype][0]-fp[stype]
        denom[denom == 0.] = -1.
        pout_ind = np.concatenate([[True], denom > 0., [True]])
        pout = np.concatenate([[0.], (fp[stype][0]-fp[stype])/denom, [.5]])
        results[stype][mtype] = 100.*(np.trapz(pout[pout_ind], 1.-fpr[pout_ind]))
        #if verbose:
           # print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
           # print('')
    
    return results


#Train

In [32]:
def test1(net, criterion, testloader, outloader, epoch=None, **options):

    net.eval()
    correct, total = 0, 0

    torch.cuda.empty_cache()

    _pred_k, _pred_u, _labels = [], [], []

    with torch.no_grad():
        for data, labels, idx in tqdm(testloader):
            if options['use_gpu']:
                data, labels = data.cuda(), labels.cuda()
            
            with torch.set_grad_enabled(False):
                x, y = net(data, True)
                logits, _ = criterion(x, y)
                predictions = logits.data.max(1)[1]
                total += labels.size(0)
                correct += (predictions == labels.data).sum()

                if options['use_softmax_in_eval']:
                    logits = torch.nn.Softmax(dim=-1)(logits)

                _pred_k.append(logits.data.cpu().numpy())
                _labels.append(labels.data.cpu().numpy())

        for batch_idx, (data, labels, idx) in enumerate(tqdm(outloader)):
            if options['use_gpu']:
                data, labels = data.cuda(), labels.cuda()
            
            with torch.set_grad_enabled(False):

                x, y = net(data, True)

                logits, _ = criterion(x, y)

                if options['use_softmax_in_eval']:
                    logits = torch.nn.Softmax(dim=-1)(logits)

                _pred_u.append(logits.data.cpu().numpy())

    # Accuracy
    acc = float(correct) * 100. / float(total)
    print('Acc: {:.5f}'.format(acc))

    _pred_k = np.concatenate(_pred_k, 0)
    _pred_u = np.concatenate(_pred_u, 0)
    _labels = np.concatenate(_labels, 0)
    
    # Out-of-Distribution detction evaluation
    x1, x2 = np.max(_pred_k, axis=1), np.max(_pred_u, axis=1)
    results = evaluation.metric_ood(x1, x2)['Bas']
    
    # OSCR
    _oscr_socre = evaluation.compute_oscr(_pred_k, _pred_u, _labels)

    # Average precision
    ap_score = average_precision_score([0] * len(_pred_k) + [1] * len(_pred_u),
                                       list(-np.max(_pred_k, axis=-1)) + list(-np.max(_pred_u, axis=-1)))

    results['ACC'] = acc
    results['OSCR'] = _oscr_socre * 100.
    results['AUPR'] = ap_score * 100

    return results,x1,x2

In [33]:
acc=[]
auroc=[]

In [34]:
for epoch in range(600):
        print("==> Epoch {}/{}".format(epoch+1, options['max_epoch']))

        train(net, criterion, optimizer, trainloader, epoch=epoch, **options)
        print("==> Test", options['loss'])
        results,x1,x2 = test1(net, criterion, testloader, outloader, epoch=epoch, **options)
        print("Epoch {}: Acc (%): {:.3f}\t AUROC (%): {:.3f}\t OSCR (%): {:.3f}\t".format(epoch,
                                                                                              results['ACC'],
                                                                                              results['AUROC'],
                                                                                              results['OSCR']))
        acc.append(results['ACC'])
        auroc.append(results['AUROC'])


  0%|          | 0/235 [00:00<?, ?it/s]

==> Epoch 1/50


  0%|          | 0/4 [00:00<?, ?it/s]

Batch 235/235	 Loss 1.589862 (1.674972)
==> Test Softmax


  0%|          | 0/235 [00:00<?, ?it/s]

...

Acc: 94.00000
Epoch 598: Acc (%): 94.000	 AUROC (%): 89.889	 OSCR (%): 87.087	
==> Epoch 600/50


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 0/4 [00:00<?, ?it/s]

Batch 235/235	 Loss 0.370751 (0.341081)
==> Test Softmax


100%|██████████| 4/4 [00:00<00:00,  4.38it/s]

Acc: 94.75000
Epoch 599: Acc (%): 94.750	 AUROC (%): 90.962	 OSCR (%): 88.515	


In [35]:
id=[]
id.append("epoch")
id

['epoch']

In [36]:
for i in range(600):
  id.append(i+1)

In [37]:
auroc.insert(0,"auroc")

In [38]:
auroc[:10]

['auroc',
 61.29500000000001,
 65.15875000000001,
 69.8025,
 70.28562500000001,
 71.67750000000001,
 72.56125000000002,
 71.091875,
 77.95625000000001,
 81.53750000000002]

In [ ]:
acc.insert(0,"acc")

In [ ]:
acc.pop(0)
auroc.pop(0)

'auroc'

In [ ]:
import pandas as pd

In [39]:
results,x1,x2 = test1(net, criterion, testloader, outloader, epoch=epoch, **options)

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 4/4 [00:01<00:00,  3.69it/s]

Acc: 94.75000


In [45]:
x1.shape

(400,)

In [46]:
c = np.concatenate((x1,x2))

In [47]:
len(c)

800

In [55]:
labe=[]

In [56]:
for i in range(800):
  if i<400:
    labe.append('1')
  else:
    labe.append('-1')


In [57]:
import matplotlib.pyplot as plt


In [58]:
from sklearn.metrics import roc_curve


# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(labe, c)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

ValueError: ignored